# Parking

In [1]:
from pymprog import begin, var, maximize, solve, end, iprod, par, minimize, vobj, solver, glpk, sensitivity
from time import clock
import random
from numpy import shape
from itertools import permutations
import csv


class TCM(object):
    
    def __init__(self,name,t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb,pr=0):
        '''t-tablica czasow trwania
        tco - sprzezenia czasowe pomiedzy obiektami
        tcb - sprzezenia czasowe pomiedzy brygadami
        wco - wagi sprzezen pomiedzy obiektami
        wcb - wagi sprzezn pomiedzy brygadami
        '''
        #const        
        self.ro=0.0001
        
        #param
        self.name=name
        
        self.t=t
        
        self.tcdo=tcdo
        self.tcdb=tcdb
        
        self.tcgo=tcgo
        self.tcgb=tcgb
        
        self.cwdo=cwdo
        self.cwdb=cwdb
        
        self.cwgo=cwgo
        self.cwgb=cwgb
        
        (self.onumber,self.bnumber)=shape(self.t)
        
        (self.time,self.FC)=self.solve(pr)

    def raport_p(self):
        print(self.name,":",self.time," FC:",self.FC )
    
    def raport(self):
        return([self.name,self.time,self.FC])
    
    def solve(self,pr):
        time=clock()
        p=begin('Parking')
        W=iprod(range(self.onumber),range(self.bnumber)) # tablica zamiennych decyzyjnych dla NWR NWZ NPR NPZ
        O=iprod(range(self.onumber-1),range(self.bnumber)) #tablica zmiennych dla cko - sprzezen korygujacych pomiedzy obiektami
        B=iprod(range(self.onumber),range(self.bnumber-1)) #tablica zmiennych dla ckb - sprzezen korygujacych pomiedzy brygadami
        
        NWR=var('NWR',W) #czas trwania czynnosci
        NWZ=var('NWZ',W)
        ckdo=var('ckdo',O)
        ckdb=var('ckdb',B)
        ckgo=var('ckgo',O)
        ckgb=var('ckgb',B)
        
        t=par('t',self.t)
        
        tcdo=par('tcdo',self.tcdo) #parametry
        tcdb=par('tcdb',self.tcdb)
        
        tcgo=par('tcgo',self.tcgo)
        tcgb=par('tcgb',self.tcgb)
        
        cwdo=par('cwdo',self.cwdo)
        cwdb=par('cwdb',self.cwdb)
        
        cwgo=par('cwgo',self.cwgo)
        cwgb=par('cwgb',self.cwgb)
        

        #funkcja celu - minimalne koszty
        minimize(sum([ckdo[i,j]*cwdo[i][j]+ckgo[i,j]*cwgo[i][j] for (i,j) in O])+sum([ckdb[i,j]*cwdb[i][j]+ckgb[i,j]*cwgb[i][j] for (i,j) in B])+NWZ[self.onumber-1,self.bnumber-1])
        
        NWR[0,0] == 0
        
        for (i,j) in W:
            NWZ[i,j]==NWR[i,j]+t[i][j]
            
        for (i,j) in O:
            NWR[i+1,j]>=NWZ[i,j]+tcdo[i][j]-ckdo[i,j]      
            NWR[i+1,j]<=NWZ[i,j]+tcgo[i][j]+ckgo[i,j]
            
        for (i,j) in B:
            NWR[i,j+1]>=NWZ[i,j]+tcdb[i][j]-ckdb[i,j]    
            NWR[i,j+1]<=NWZ[i,j]+tcgb[i][j]+ckgb[i,j]
                       
        #parametry solvera
        #solver(float,msg_lev=glpk.GLP_MSG_OFF)
        
        p.solve()
        
        if pr:
            print('FC:',vobj())

            for (i,j) in W:
                print('O',i+1,' B',j+1,'NWR',NWR[i,j].primal,'NWZ',NWZ[i,j].primal)
            for (i,j) in O:
                print(i+1,j+1,'ckdo',ckdo[i,j].primal,'ckgo',ckgo[i,j].primal)       
            for (i,j) in B:
                print(i+1,j+1,'ckdb',ckdb[i,j].primal,'ckgb',ckgb[i,j].primal)
            print('Termin realizacji:',NWZ[self.onumber-1,self.bnumber-1].primal)

            print('Czas kompilacji:',round(clock()-time,4),'s')
        
        return NWZ[self.onumber-1,self.bnumber-1].primal,vobj()
    
        #sensitivity()

# Model A.8a: Harmonogramowanie priorytetowe
Kolejność
B3
O2
B2

In [18]:
t=[[19,26,38,22,18,9],
   [15,18,35,14,8,5],
   [13,18,32,13,8,5],
   [14,22,33,16,10,7],
   [15,17,28,12,8,5]]
tc=t
tcdo=[[0,0,0,0,0,0],
     [0,0,0,0,0,0],
     [0,0,0,0,0,0],
     [0,0,0,0,0,0],]
tcdb=[[0,0,0,0,0],
     [0,0,0,0,0],
     [0,0,0,0,0],
     [0,0,0,0,0],
     [0,0,0,0,0]]
    
tcgo=[[100000000,100000000,100000000,0,100000000,100000000],
     [100000000,100000000,100000000,0,100000000,100000000],
     [100000000,100000000,100000000,0,100000000,100000000],
     [100000000,100000000,100000000,0,100000000,100000000],]
tcgb=[[100000000,100000000,100000000,0,100000000],
     [100000000,100000000,100000000,0,100000000],
     [100000000,100000000,100000000,0,100000000],
     [100000000,100000000,100000000,0,100000000],
     [100000000,100000000,100000000,0,100000000]]
    
cwdo=[[100000000,100000000,100000000,100000000,100000000,100000000],
     [100000000,100000000,100000000,100000000,100000000,100000000],
     [100000000,100000000,100000000,100000000,100000000,100000000],
     [100000000,100000000,100000000,100000000,100000000,100000000],]
cwdb=[[100000000,100000000,100000000,100000000,100000000],
     [100000000,100000000,100000000,100000000,100000000],
     [100000000,100000000,100000000,100000000,100000000],
     [100000000,100000000,100000000,100000000,100000000],
     [100000000,100000000,100000000,100000000,100000000]]
    
cwgo=[[100,100,100,100000000,100,100],
     [100,100,100,100000000,100,100],
     [100,100,100,100000000,100,100],
     [100,100,100,100000000,100,100],]   
cwgb=[[100,100,100,100000,100],
     [100,100,100,100000,100],
     [100,100,100,100000,100],
     [100,100,100,100000,100],
     [100,100,100,100000,100]]


P=TCM("Parking",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb,1)

FC: 400236.0
O 1  B 1 NWR 0.0 NWZ 19.0
O 1  B 2 NWR 19.0 NWZ 45.0
O 1  B 3 NWR 45.0 NWZ 83.0
O 1  B 4 NWR 146.0 NWZ 168.0
O 1  B 5 NWR 168.0 NWZ 186.0
O 1  B 6 NWR 186.0 NWZ 195.0
O 2  B 1 NWR 19.0 NWZ 34.0
O 2  B 2 NWR 45.0 NWZ 63.0
O 2  B 3 NWR 83.0 NWZ 118.0
O 2  B 4 NWR 168.0 NWZ 182.0
O 2  B 5 NWR 186.0 NWZ 194.0
O 2  B 6 NWR 195.0 NWZ 200.0
O 3  B 1 NWR 34.0 NWZ 47.0
O 3  B 2 NWR 63.0 NWZ 81.0
O 3  B 3 NWR 118.0 NWZ 150.0
O 3  B 4 NWR 182.0 NWZ 195.0
O 3  B 5 NWR 195.0 NWZ 203.0
O 3  B 6 NWR 203.0 NWZ 208.0
O 4  B 1 NWR 47.0 NWZ 61.0
O 4  B 2 NWR 81.0 NWZ 103.0
O 4  B 3 NWR 150.0 NWZ 183.0
O 4  B 4 NWR 195.0 NWZ 211.0
O 4  B 5 NWR 211.0 NWZ 221.0
O 4  B 6 NWR 221.0 NWZ 228.0
O 5  B 1 NWR 61.0 NWZ 76.0
O 5  B 2 NWR 103.0 NWZ 120.0
O 5  B 3 NWR 183.0 NWZ 211.0
O 5  B 4 NWR 211.0 NWZ 223.0
O 5  B 5 NWR 223.0 NWZ 231.0
O 5  B 6 NWR 231.0 NWZ 236.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
1 5 ckdo 0.0 ckgo 0.0
1 6 ckdo 0.0 ckgo 0.0
2 1 ck

In [3]:
with open('results.csv', 'w', newline='') as csvfile:
    n = csv.writer(csvfile, delimiter=" ")
    for i in zip(permutations(["P1","P2","P3","P4","P5"]),permutations(t)):
        temp=TCM(i[0],i[1],tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)
        z=temp.raport()
        n.writerow(["".join(z[0]),z[1],z[2]])

In [16]:
c=[2,0,3,1,4]
t = [ tc[i] for i in c]
print(tc)
P=TCM("Parking",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb,1)

[[19, 26, 38, 22, 18, 9], [15, 18, 35, 14, 8, 5], [13, 18, 32, 13, 8, 5], [14, 22, 33, 16, 10, 7], [15, 17, 28, 12, 8, 5]]
FC: 200222.0
O 1  B 1 NWR 0.0 NWZ 13.0
O 1  B 2 NWR 13.0 NWZ 31.0
O 1  B 3 NWR 31.0 NWZ 63.0
O 1  B 4 NWR 132.0 NWZ 145.0
O 1  B 5 NWR 145.0 NWZ 153.0
O 1  B 6 NWR 153.0 NWZ 158.0
O 2  B 1 NWR 13.0 NWZ 32.0
O 2  B 2 NWR 32.0 NWZ 58.0
O 2  B 3 NWR 63.0 NWZ 101.0
O 2  B 4 NWR 145.0 NWZ 167.0
O 2  B 5 NWR 167.0 NWZ 185.0
O 2  B 6 NWR 185.0 NWZ 194.0
O 3  B 1 NWR 32.0 NWZ 46.0
O 3  B 2 NWR 58.0 NWZ 80.0
O 3  B 3 NWR 101.0 NWZ 134.0
O 3  B 4 NWR 167.0 NWZ 183.0
O 3  B 5 NWR 185.0 NWZ 195.0
O 3  B 6 NWR 195.0 NWZ 202.0
O 4  B 1 NWR 46.0 NWZ 61.0
O 4  B 2 NWR 80.0 NWZ 98.0
O 4  B 3 NWR 134.0 NWZ 169.0
O 4  B 4 NWR 183.0 NWZ 197.0
O 4  B 5 NWR 197.0 NWZ 205.0
O 4  B 6 NWR 205.0 NWZ 210.0
O 5  B 1 NWR 61.0 NWZ 76.0
O 5  B 2 NWR 98.0 NWZ 115.0
O 5  B 3 NWR 169.0 NWZ 197.0
O 5  B 4 NWR 197.0 NWZ 209.0
O 5  B 5 NWR 209.0 NWZ 217.0
O 5  B 6 NWR 217.0 NWZ 222.0
1 1 ckdo 0.0 ckgo

In [17]:
c=[2,1,3,4,0]
t = [ tc[i] for i in c]
print(tc)
P=TCM("Parking",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb,1)

[[19, 26, 38, 22, 18, 9], [15, 18, 35, 14, 8, 5], [13, 18, 32, 13, 8, 5], [14, 22, 33, 16, 10, 7], [15, 17, 28, 12, 8, 5]]
FC: 246.0
O 1  B 1 NWR 0.0 NWZ 13.0
O 1  B 2 NWR 13.0 NWZ 31.0
O 1  B 3 NWR 31.0 NWZ 63.0
O 1  B 4 NWR 142.0 NWZ 155.0
O 1  B 5 NWR 155.0 NWZ 163.0
O 1  B 6 NWR 163.0 NWZ 168.0
O 2  B 1 NWR 13.0 NWZ 28.0
O 2  B 2 NWR 31.0 NWZ 49.0
O 2  B 3 NWR 63.0 NWZ 98.0
O 2  B 4 NWR 155.0 NWZ 169.0
O 2  B 5 NWR 169.0 NWZ 177.0
O 2  B 6 NWR 177.0 NWZ 182.0
O 3  B 1 NWR 28.0 NWZ 42.0
O 3  B 2 NWR 49.0 NWZ 71.0
O 3  B 3 NWR 98.0 NWZ 131.0
O 3  B 4 NWR 169.0 NWZ 185.0
O 3  B 5 NWR 185.0 NWZ 195.0
O 3  B 6 NWR 195.0 NWZ 202.0
O 4  B 1 NWR 42.0 NWZ 57.0
O 4  B 2 NWR 71.0 NWZ 88.0
O 4  B 3 NWR 131.0 NWZ 159.0
O 4  B 4 NWR 185.0 NWZ 197.0
O 4  B 5 NWR 197.0 NWZ 205.0
O 4  B 6 NWR 205.0 NWZ 210.0
O 5  B 1 NWR 57.0 NWZ 76.0
O 5  B 2 NWR 88.0 NWZ 114.0
O 5  B 3 NWR 159.0 NWZ 197.0
O 5  B 4 NWR 197.0 NWZ 219.0
O 5  B 5 NWR 219.0 NWZ 237.0
O 5  B 6 NWR 237.0 NWZ 246.0
1 1 ckdo 0.0 ckgo 0.0
